In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import warnings
# Disable pysindy to generate a deprecation warning
warnings.filterwarnings("ignore", message="pkg_resources is deprecated")
import pysindy as ps
import scipy as sp
import os
from sklearn.metrics import mean_squared_error
import pickle
from matplotlib import gridspec
import pandas as pd
from scipy import stats
import matplotlib.legend_handler
from utils import Dataset

In [ ]:
params = { 'figure.figsize': (15.,5.),
          'axes.labelsize': 22,
          'xtick.labelsize': 16,
          'ytick.labelsize': 16,
          'legend.fontsize': 16,
          # 'font.sans-serif': 'Arial', # RB: This does not work on unix wihtout fonts installed
          'lines.linewidth': 2}
plt.rcParams.update(params)

In [ ]:
colors = ["#377eb8", "#ff7f00", "#4daf4a"]

## Read the data of AVM patients

In [ ]:
# folderpath = r"../data/clinical_data/"
# filepaths = [os.path.join(folderpath, name) for name in os.listdir(folderpath)]
# patients = []
# for i in range(len(filepaths)):
#     patients.append(str.split(os.path.basename(filepaths[i]), '-')[0])
# patients = np.unique(patients)

In [ ]:
# patients_AA = ['G2', 'K1', 'K3', 'R1', 'S2']
# patients_AVM = ['G1', 'K2', 'L1', 'S1', 'S3']

In [ ]:
dataset = Dataset('data/AVM')
P = dataset.all_pressures()
V = dataset.all_velocities()
# for i in range(len(filepaths)):
#     if (str.split(os.path.basename(filepaths[i]), '-')[1] == 'AVM'):
#         P[i] = np.loadtxt(filepaths[i]+"/data_p.txt")
#         V[i] = np.loadtxt(filepaths[i]+"/data_v.txt")

In [ ]:
# for i in P.keys():
#     P[i] = P[i]*100 #pressure in mmHg

In [ ]:
fig, axs = plt.subplots(ncols=1, nrows=5, sharex=True, sharey=True, figsize=(10,4))
axs = axs.ravel()
for idx, i in enumerate(list(P.keys())[:5]):
    axs[idx].plot(P[i])

In [ ]:
def Heun(f, y0, t):  # Heun integrator
    y = np.zeros((len(t),len(y0)))
    y[0] = y0
    for i in range(len(t)-1):
        dt = t[i+1] - t[i]
        y_tilde = y[i] + dt*f(y[i],t[i],v[i]) # Euler formulation
        y[i+1] = y[i] + 0.5*dt*(f(y[i],t[i],v[i])+f(y_tilde,t[i]+dt,v[i+1]))
    return y

In [ ]:
dt = 1/200

## state variables

In [ ]:
# p and pdot are the state variables for a second order ODE model discovery
states = {}
for i in P.keys():
    states[i] = np.vstack([P[i], np.gradient(P[i], dt), V[i]])

## construct the candidate library

In [ ]:
lib_V = ps.PolynomialLibrary(degree=1, include_bias=False, include_interaction=False)
lib_P = ps.PolynomialLibrary(degree=3, include_bias=False, include_interaction=True)
lib = ps.GeneralizedLibrary([lib_P, lib_V], inputs_per_library=np.array([[0,1],[2,2]]))

In [ ]:
feature_names = ["p", "p'", "v"]

## calculate the numerical derivatives

In [ ]:
#for the left hand side of the ODEs, we need pdot and pdotdot
derivatives = {}
for i in P.keys():
    derivatives[i] = np.zeros_like(states[i])
    for j in range(3):
        derivatives[i][j] = np.gradient(states[i][j,:], dt)

## model discovery

In [ ]:
scikit_optimizer = ps.STLSQ(threshold=.1, alpha=0., fit_intercept=False)
coeff = {}
for i in P.keys():
    model = ps.SINDy(optimizer=scikit_optimizer, feature_library=lib, 
                        feature_names=feature_names).fit(states[i].T, t=dt, x_dot=derivatives[i][:2].T, quiet=True)
    coeff[i] = model.coefficients()
    model.print(lhs=['dotp', 'ddotp', 'v'])

## simulations of predicted models

In [ ]:
def predicted_pressure(u,t,v):
    return np.array([u[1],
                     coef[0]*u[0] + coef[1]*u[1] + coef[2]*u[0]**2 + coef[3]*u[0]*u[1] + coef[4]*u[1]**2 
                     + coef[5]*u[0]**3 + coef[6]*(u[0]**2)*u[1] + coef[7]*u[0]*(u[1]**2) + coef[8]*u[1]**3 + coef[9]*v])

In [ ]:
import warnings
# Some models, which may cause overflow, are discarded from candicates
warnings.filterwarnings("ignore", message="overflow encountered in scalar power")
warnings.filterwarnings("ignore", message="invalid value encountered in scalar multiply")

predicted_p = {}
for i in P.keys():
    v = V[i] #include the emprical velocity in the simulations of predicted models
    coef = coeff[i][1]
    time = np.array([j*dt for j in range(len(P[i]))])
    predicted_p[i] = Heun(predicted_pressure, [P[i][0], 0.], time) #initial value is chosen from the empirical pressure

In [ ]:
non_problematic_keys = ['K2.BEFORE', 'K2.DURING', 'K2.AFTER', 'G1.BEFORE', 'S3.DURING', 'S3.AFTER', 'L1.BEFORE', 
                         'L1.DURING', 'L1.AFTER', 'S1.BEFORE', 'S1.DURING', 'S1.AFTER']


## comparison between the simulated pressure and empirical pressure

In [ ]:
fig, axs = plt.subplots(nrows=12, ncols=1, figsize=(10,16), sharex=True)
axs = axs.ravel()
for idx, i in enumerate(non_problematic_keys):
    axs[idx].plot(np.array([k*dt for k in range(len(P[i]))]), P[i], label="Experimental pressure")
    axs[idx].plot(np.array([k*dt for k in range(len(P[i]))]), predicted_p[i][:,0], label="Simulated pressure")
    
axs[11].set_xlabel("Time (s)")
axs[5].set_ylabel("Pressure (mmHg)")
axs[0].legend(frameon=False, fontsize=12)
fig.tight_layout()